# PatchMatch Evaluation on VO (Visual Odometry) Dataset

### Import Required Libraries and Packages

In [9]:
import os
import glob
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
module_path = os.path.abspath(os.path.join('..', 'modules'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from patchmatch import PatchMatch

### ORB Matching Pipeline Evaluation

In [26]:
folder_path = os.path.abspath(os.path.join('..', 'media/sequence_50/images'))
image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
image_files.sort()

image_pairs = []
for i in range(200): # len(image_files) - 1):
    img1_path = os.path.join(folder_path, image_files[i])
    image_1 = cv2.imread(img1_path)
    image_1_gray = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
        
    img2_path = os.path.join(folder_path, image_files[i + 1])
    image_2 = cv2.imread(img2_path)
    image_2_gray = cv2.cvtColor(image_2, cv2.COLOR_BGR2GRAY)

    orb = cv2.ORB_create(nfeatures=1000)
    keypoints1, descriptors1 = orb.detectAndCompute(image_1_gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(image_2_gray, None)

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    matches = bf.match(descriptors1, descriptors2)

    matches = sorted(matches, key = lambda x:x.distance)

    points1 = []
    points2 = []

    for match in matches:
        points1.append(keypoints1[match.queryIdx].pt)
        points2.append(keypoints2[match.trainIdx].pt)

    points1 = np.array(points1, dtype=np.int32)
    points2 = np.array(points2, dtype=np.int32)

    fundamental_matrix, inliers = cv2.findFundamentalMat(points1, points2, method=cv2.FM_RANSAC, ransacReprojThreshold=1.0, confidence=0.99)

    inliers = inliers.ravel().astype(bool)
    points1 = points1[inliers]
    points2 = points2[inliers]

    combined_image = np.hstack((image_1, image_2))

    for index in range(len(points1)):
        cv2.line(combined_image, (points1[index][0], points1[index][1]), (points2[index][0] + image_1.shape[1],  points2[index][1]), (0, 255, 0), 1)

    cv2.putText(combined_image, f'ORB Matching Pipeline', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 4, cv2.LINE_AA)
    cv2.putText(combined_image, f'Number of Matches: {len(points1)}', (20, 130), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 4, cv2.LINE_AA)
    
    plt.figure(figsize=(14, 8))
    plt.imshow(combined_image)
    plt.axis('off')
    plt.savefig(os.path.abspath(os.path.join('..', f'media/sequence_50/matching/orb/orb_{i}.png')), bbox_inches='tight', pad_inches=0)
    # plt.show()
    plt.close()

### PatchMatch Pipeline Evaluation

In [29]:
folder_path = os.path.abspath(os.path.join('..', 'media/sequence_50/images'))
image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
image_files.sort()

image_pairs = []
for i in range(200): # len(image_files) - 1):
    img1_path = os.path.join(folder_path, image_files[i])
    image_1 = cv2.imread(img1_path)
        
    img2_path = os.path.join(folder_path, image_files[i + 1])
    image_2 = cv2.imread(img2_path)

    pm = PatchMatch(match_feature='ORB', num_features=1000)
    points1, points2 = pm.match_two_images(image_1, image_2)

    fundamental_matrix, inliers = cv2.findFundamentalMat(points1, points2, method=cv2.FM_RANSAC, ransacReprojThreshold=1.0, confidence=0.99)

    inliers = inliers.ravel().astype(bool)
    points1 = points1[inliers]
    points2 = points2[inliers]

    combined_image = np.hstack((image_1, image_2))

    for index in range(len(points1)):
        cv2.line(combined_image, (points1[index][0], points1[index][1]), (points2[index][0] + image_1.shape[1],  points2[index][1]), (0, 255, 0), 1)

    cv2.putText(combined_image, f'PatchMatch (ORB) Matching Pipeline', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 4, cv2.LINE_AA)
    cv2.putText(combined_image, f'Number of Matches: {len(points1)}', (20, 130), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 4, cv2.LINE_AA)
    
    plt.figure(figsize=(14, 8))
    plt.imshow(combined_image)
    plt.axis('off')
    plt.savefig(os.path.abspath(os.path.join('..', f'media/sequence_50/matching/patchmatch/patchmatch_{i}.png')), bbox_inches='tight', pad_inches=0)
    # plt.show()
    plt.close()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### Create GIF from Matching Sequences

In [33]:
def make_gif(frame_folder):
    image_files = [f for f in os.listdir(frame_folder) if os.path.isfile(os.path.join(frame_folder, f))]
    image_files.sort()
    
    frames = [Image.open(os.path.join(frame_folder, image_path)) for image_path in image_files]
    
    frame_one = frames[0]
    frame_one.save(os.path.abspath(os.path.join('..', f'media/patchmatch.gif')), format="GIF", append_images=frames,
                   save_all=True, duration=100, loop=0)

In [32]:
make_gif(os.path.abspath(os.path.join('..', f'media/sequence_50/matching/orb')))

In [34]:
make_gif(os.path.abspath(os.path.join('..', f'media/sequence_50/matching/patchmatch')))